# Aims and Objectives
This study will aim to investigate the effect of funding on local school results. Two datasets will be used to determine this; one details pupils’ attainment at the end of KS2, and one contains details related to school funding.

This study will consider two questions; 
- Do local authorities allocate funding in a similar fashion to how the central government fund academies? 
- Does more funding allocation in a local authority correlate with increased average scores for the local community schools?


In [ ]:
# Import the required libraries (more added as running through notebook)

import pandas as pd
import scipy.stats
import numpy as np

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples
from collections import Counter

# First import and review of funding data

In [ ]:
# By default when importing an excel sheet, only the first sheet is imported if not sheet is called.
pd.set_option('display.max_colwidth', -1)
pd.read_excel('data/SR63_2016_Tables.xlsx')

Looking at the list of tables, a few tables that may be useful are:
- Table 6
- Table 7
- Table 8
- Table 9
- Table 10
- Table 11
- Table 12

I will import each of these tables in below to see what data they provide.

In [ ]:
# Importing table 6
pd.read_excel('data/SR63_2016_Tables.xlsx', sheetname='Table 6').head(10)

In [ ]:
# Importing table 7
pd.read_excel('data/SR63_2016_Tables.xlsx', sheetname='Table 7').head(10)

In [ ]:
# Importing table 8 -
pd.read_excel('data/SR63_2016_Tables.xlsx', sheetname='Table 8').head(10)

In [ ]:
# Importing table 9 -
pd.read_excel('data/SR63_2016_Tables.xlsx', sheetname='Table 9').head(10)

In [ ]:
# Importing table 10 -
pd.read_excel('data/SR63_2016_Tables.xlsx', sheetname='Table 10').head(10)

In [ ]:
# Importing table 11
pd.read_excel('data/SR63_2016_Tables.xlsx', sheetname='Table 11').head(10)

In [ ]:
# Importing table 12
pd.read_excel('data/SR63_2016_Tables.xlsx', sheetname='Table 12').head(10)

Looking at the above tables, the one that appears to be most relevant is table 12; I will clean up the table via OpenRefine and import in the cleaned data.

I also analysed the SFR27_2016_Main_Tables.xlsx dataset and found that the most beneficial table was the RAW Data SATs; I will also clean this table via OpenRefine.

# Importing the KS2 data
In order to import the KS2, a few steps will be required:
<br>1) Import the LEA data
<br>2) Import and clean up the KS2 data 
<br>3) Merge the LEA and KS2 data

These steps were provided by the OU in TMA02, so those steps can be reused.

## Import the LEA data
The LEA data provides all the Local Authority (LA) and region codes.

In [ ]:
leas_df = pd.read_csv('data/2015-2016/la_and_region_codes_meta.csv')
leas_df.head()

## Import and clean up the KS2 data

In [ ]:
ks2cols = pd.read_csv('data/2015-2016/ks2_meta.csv')
ks2cols['Field Name'] = ks2cols['Field Name'].apply(lambda r: r.strip(),)
ks2cols

Some columns contain int values, but pandas will treat a numeric column with na values as float64, so the int columns will need to be converted.

In [ ]:
int_cols = [c for c in ks2cols['Field Name'] 
            if c.startswith('T')
            if c not in ['TOWN', 'TELNUM', 'TKS1AVERAGE']]
int_cols += ['RECTYPE', 'ALPHAIND', 'LEA', 'ESTAB', 'URN', 'URN_AC', 'ICLOSE']
int_cols += ['READ_AVERAGE', 'GPS_AVERAGE', 'MAT_AVERAGE']

Any columns that contain percentages will be converted to floating point numbers during import.

In [ ]:
def p2f(x):
    if x.strip('%').isnumeric():
        return float(x.strip('%'))/100
    elif x in ['SUPP', 'NEW', 'LOWCOV', 'NA', '']:
        return 0.0
    else:
        return x

In [ ]:
percent_cols = [f for f in ks2cols['Field Name'] if f.startswith('P')]
percent_cols += ['WRITCOV', 'MATCOV', 'READCOV'] 
percent_cols += ['PTMAT_HIGH', 'PTREAD_HIGH', 'PSENELSAPK', 'PSENELK', 'PTGPS_HIGH']
percent_converters = {c: p2f for c in percent_cols}

Inital import of the KS2 data.

In [ ]:
ks2_df = pd.read_csv('data/2015-2016/england_ks2final.csv', 
                   na_values=['SUPP', 'NEW', 'LOWCOV', 'NA', ''],
                   converters=percent_converters)

In [ ]:
sorted(ks2_df['RECTYPE'].unique())

5 record types exist - Record type (1=mainstream school; 2=special school; 3=Local Authority; 4=National (all schools); 5=National (maintained schools))<br>
The only ones required are 1 and 2, so the other record types can be dropped.

In [ ]:
ks2_df = ks2_df[(ks2_df['RECTYPE'] == 1) | (ks2_df['RECTYPE'] == 2)]

All values should be converted to numbers where possible.

In [ ]:
ks2_df = ks2_df.apply(pd.to_numeric, errors='ignore')

## Merge the LEA and KS2 data

In [ ]:
ks2_df = pd.merge(ks2_df, leas_df, on=['LEA'])
ks2_df.head().T

# KS2 data analysis

The first step I will take to analyse the data is to identify the different school types, and the counts of each type.

In [ ]:
# Checked via Excel - rows 3 to 26 are relevant
school_types_df = pd.read_excel('data/2015-2016/abbreviations.xlsx')[3:26]
school_types_df.drop('Unnamed: 2', axis=1, inplace=True)
school_types_df.rename(columns={ school_types_df.columns[0]: "NFTYPE", school_types_df.columns[1]: 'Description'} , inplace=True)
school_types_df

In [ ]:
school_types_df.to_csv('data/school_types.csv')

In [ ]:
ks2_df['NFTYPE'].value_counts()

In order to identify which school types are funded by Local Authorities and which are funded by Central Government, I searched various online resources:<br>
<br>http://www.bbc.co.uk/schools/parents/types_of_schools/
<br>https://www.gov.uk/types-of-school/city-technology-colleges
<br>https://www.newschoolsnetwork.org/set-up-a-free-school/frequently-asked-questions/how-are-free-schools-funded
<br>https://en.wikipedia.org/wiki/University_technical_college

Each school type has been placed into a FundingBody category. These categories may change as I perform further investigation. 

In [ ]:
st_fb_df = pd.read_csv('data/school_types_with_funding_bodies.csv')
st_fb_df

Next I will identify the school types that appear in the KS2 dataset, as that should narrow down the number of school types.

In [ ]:
ks2_st_df = ks2_df['NFTYPE'].unique()

ks2_st_list = ks2_st_df.tolist()

ks2_st_list

In [ ]:
ks2_st_fb_df = st_fb_df.loc[st_fb_df['NFTYPE'].isin(ks2_st_list)]

ks2_st_fb_df

In [ ]:
print("st_fb_df row count: " + str(len(st_fb_df)))
print("ks2_st_fb_df row count: " + str(len(ks2_st_fb_df)))

# KS2 final review and import

I have identified the columns that I would require from the ks2_df dataset in order for the questions to be answered:
- RECTYPE - this is the type of record; only values 1 or 2 are relevant as they refer to mainstream and special schools (KS2 dataset)
- AILPHAIND - Alphabetic Index (KS2 dataset)
- LEA - Local Authority number (KS2 dataset)
- ESTAB - Establishment number (KS2 dataset)
- URN - School UID (KS2 dataset)
- SCHNAME - School name (kS2 dataset)
- NFTYPE - School type (KS2 dataset)
- READ_AVERAGE - Reading average (KS2 dataset)
- GPS_AVERAGE - Grammar, punctuation and spelling average
- MAT_AVERAGE - Maths average
- LA Name - Local Authority Name (LA and Region codes meta dataset)
- REGION - Region number (LA and Region codes meta dataset)
- REGION NAME - Region name (LA and Region codes meta dataset)

In [ ]:
ks2_df = ks2_df[['RECTYPE', 'ALPHAIND', 'LEA', 'ESTAB', 'URN', 'SCHNAME', 'NFTYPE', 'READ_AVERAGE', 'GPS_AVERAGE', 'MAT_AVERAGE', 'LA Name', 'REGION', 'REGION NAME']]

#Drop any rows that contain NA values
ks2_df.dropna(inplace=True)

int_cols = ['RECTYPE', 'ALPHAIND', 'LEA', 'ESTAB', 'URN', 'READ_AVERAGE', 'GPS_AVERAGE', 'MAT_AVERAGE', 'REGION']

for col in int_cols:
    ks2_df[col] = ks2_df[col].astype(int)


ks2_df

# Second import and review of funding data

I cleaned the funding datasets via OpenRefine so it could be imported easier into the notebook.

In [ ]:
sr63_df = pd.read_csv('data/SR63_T12_cleaned.csv')

len(sr63_df)

sr63_df.head()

for k in sr63_df.keys():
    print(k)

In [ ]:
sfr27_df = pd.read_csv('data/SFR27_RDS_cleaned.csv')

len(sfr27_df)

sfr27_df.head()

for k in sfr27_df.keys():
    print(k)

The KS2 dataset and two funding datasets can be combined through a concatenation of the LA number and Estab number; 
I will create a common field between the three datasets called _LAESTAB

In [ ]:
# Create _LAESTAB column on ks2_df to allow join to funding data
ks2_df['_LAESTAB'] = ks2_df['LEA'].map(str) + ks2_df['ESTAB'].map(str)

# Convert _LAESTAB to int
ks2_df['_LAESTAB'] = ks2_df['_LAESTAB'].astype(int)

ks2_df.head().T

In [ ]:
# Create LA-ESTAB column on sfr27_df
sfr27_df['_LAESTAB'] = sfr27_df['LA'].map(str) + sfr27_df['Estab'].map(str)

# Convert _LAESTAB to int
sfr27_df['_LAESTAB'] = sfr27_df['_LAESTAB'].astype(int)

sfr27_df.head()

In [ ]:
# Rename LA/ESTAB number column on sr63_df to _LAESTAB
sr63_df.rename(columns={'LA/ESTAB number' : '_LAESTAB', 'Total revenue balance (1)': 
                                    'Total revenue balance'}, inplace=True)

sr63_df.head()

The two balance fields from the funding datasets appear to have different scales for the amount; to match them, I will multiply the <strong>Total Grant funding £000</strong> and <strong>Total Income £000</strong> sfr27_df dataframe by 1000.

In [ ]:
sfr27_df['_TotalGrantFunding'] = sfr27_df['Total Grant Funding  £000'] * 1000
sfr27_df['_TotalIncome'] = sfr27_df['Total Income  £000'] * 1000
sfr27_df.head()

In [ ]:
query = '''
SELECT
    ks2.*,
    COALESCE(sr63."Total revenue balance", sfr27."_TotalGrantFunding") AS _BALANCE
FROM
    ks2_df ks2
    LEFT JOIN sr63_df  sr63  ON ks2."_LAESTAB" =  sr63."_LAESTAB"
    LEFT JOIN sfr27_df sfr27 ON ks2."_LAESTAB" = sfr27."_LAESTAB"
'''

ks2_funding_df = pysqldf(query)

# Remove rows with NaN value for _BALANCE
ks2_funding_df.dropna(inplace=True)
ks2_funding_df.head().T

Storing ks2_funding_df in new variable - ks2_funding2_df

In [ ]:
query = '''
SELECT
    RECTYPE,
    ALPHAIND,
    LEA,
    ESTAB,
    URN,
    SCHNAME,
    NFTYPE,
    READ_AVERAGE,
    GPS_AVERAGE,
    MAT_AVERAGE,
    "LA Name",
    REGION,
    "REGION NAME",
    _LAESTAB,
    CASE
    WHEN _BALANCE LIKE '%,%' THEN REPLACE(_BALANCE, ',', '')
    ELSE _BALANCE
    END AS _BALANCE
FROM
    ks2_funding_df
'''

ks2_funding2_df = pysqldf(query)
ks2_funding2_df.head()

In [ ]:
ks2_funding2_df['_BALANCE'] = ks2_funding2_df['_BALANCE'].apply(pd.to_numeric, errors='coerce')

ks2_funding2_df.head()

In [ ]:
query = '''
SELECT DISTINCT
NFTYPE
FROM
ks2_funding2_df'''

pysqldf(query)

In [ ]:
query = '''
SELECT
    *
FROM
    ks2_funding2_df
WHERE
    _BALANCE IS NOT NULL AND _BALANCE > 0
'''

ks2_funding2_df = pysqldf(query)
ks2_funding2_df.sort_values(by=['_BALANCE'],ascending=False)

In [ ]:
query = '''
SELECT DISTINCT
NFTYPE
FROM
ks2_funding2_df
'''

pysqldf(query)

In [ ]:
query = '''
SELECT
    NFTYPE,
    COUNT(*) AS _SCHOOLCOUNT,
    AVG(_BALANCE) AS _AVGBALANCE,
    MAX(_BALANCE) AS _MAXBALANCE,
    MIN(_BALANCE) AS _MINBALANCE
FROM
    ks2_funding2_df
GROUP by
    NFTYPE'''

ks2_funding_avg_df = pysqldf(query)
ks2_funding_avg_df

In [ ]:
ks2_funding2_avg_df = pd.merge(ks2_funding_avg_df, ks2_st_fb_df, how="inner", on="NFTYPE")

ks2_funding2_avg_df

In [ ]:
ks2_funding2_avg_df['_AVGBALANCE'] = ks2_funding2_avg_df['_AVGBALANCE'] / 1000

ks2_funding2_avg_df

In [ ]:
ks2_funding2_avg_df.rename(columns={"_AVGBALANCE" : "AVGBALANCE (000s)"}, inplace=True)

ks2_funding2_avg_df['AVGBALANCE (000s)'] = ks2_funding2_avg_df['AVGBALANCE (000s)'].astype(int)

ks2_funding2_avg_df

In [ ]:
ks2_funding2_avg_df['AVGBALANCE (000s)'] = ks2_funding2_avg_df['AVGBALANCE (000s)'] * 1000

In [ ]:
ks2_funding2_avg_df.rename(columns={"AVGBALANCE (000s)" : "_AVGBALANCE"}, inplace=True)
ks2_funding2_avg_df.sort_values(by=['_AVGBALANCE'],ascending=False, inplace=True)
ks2_funding2_avg_df

In [ ]:
ks2_funding2_avg_bar = ks2_funding2_avg_df.plot.bar(
    x='Description',
    y='_AVGBALANCE',
    color=[np.where(ks2_funding2_avg_df['FundingBody']=='Central Government', 'red', 'blue')],
    title='Average Balance Per School Type',
    legend=False,
    figsize=(10, 7)
    )

for p in ks2_funding2_avg_bar.patches:
    ks2_funding2_avg_bar.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

In [ ]:
ks2_funding2_df = pd.merge(ks2_funding2_df, ks2_st_fb_df, how="inner", on="NFTYPE")
ks2_funding2_df.head()

In [ ]:
query = '''
SELECT
    SCHNAME,
    Description,
    FundingBody,
    _BALANCE
FROM
    ks2_funding2_df
ORDER BY
    _BALANCE DESC
LIMIT 10
'''

pysqldf(query)

In [ ]:
query = '''
SELECT
    SCHNAME,
    Description,
    FundingBody,
    _BALANCE
FROM
    ks2_funding2_df
ORDER BY
    _BALANCE ASC
LIMIT 10
'''

pysqldf(query)

<h1>Do local authorities allocate funding in a similar fashion to how the central government fund academies?</h1>
Looking at both the bar chart and the tables showing the top and bottom 30 balances from the ks2_funding2_df DataFrame, it appears that the Central Government allocate much higher funding to Academies than Local Authorities allocate to Community, Foundation and Voluntary schools. One reason for the higher funding may be due to Academies not being required to follow the national curriculum, so may provide other subjects that require higher funding (https://fullfact.org/education/academies-and-maintained-schools-what-do-we-know/). Another reason could be the size of Academies are generally larger than other school types, so higher running costs may exist.

# KS2 and funding comparison

Next step  is to find if the funding allocated affects the results; I will attempt to use data mining to do this.

In [ ]:
ks2_funding3_df = ks2_funding2_df.copy()

ks2_funding3_df

In [ ]:
query='''
SELECT
    READ_AVERAGE,
    _BALANCE
FROM
    ks2_funding3_df
ORDER BY
    1
'''

read_avg_bal_df = pysqldf(query)

read_avg_bal_df.plot.scatter(
    x='READ_AVERAGE', 
    y='_BALANCE',
    title='READ AVERAGE PER BALANCE')

Looking at the basic graph containing the read average per balance, it appears there is a almost perfect bell curve with the data; I will investigate this further by clustering the data to see if any patterns appear.

In [ ]:
query = '''
SELECT
    READ_AVERAGE,
    COUNT(*)
FROM
    ks2_funding3_df
GROUP BY
    READ_AVERAGE
ORDER BY
    1
'''

pysqldf(query)

In [ ]:
#http://code.activestate.com/recipes/425397-split-a-list-into-roughly-equal-sized-pieces/
def split_seq(seq, size):
        newseq = []
        splitsize = 1.0/size*len(seq)
        for i in range(size):
                newseq.append(seq[int(round(i*splitsize)):int(round((i+1)*splitsize))])
        return newseq

In [ ]:
query = '''
SELECT DISTINCT
    READ_AVERAGE
FROM
    ks2_funding3_df
ORDER BY
    1
'''

ra_df = pysqldf(query)

split_seq(ra_df.values.tolist(),3)

In [ ]:
query = '''
SELECT
    CASE
        WHEN READ_AVERAGE BETWEEN 89 AND 98 THEN "LOW"
        WHEN READ_AVERAGE BETWEEN 99 AND 106 THEN "MEDIUM"
        WHEN READ_AVERAGE > 106 THEN "HIGH"
        ELSE NULL
    END AS CAT,
    COUNT(*)
FROM
    ks2_funding3_df
GROUP BY
    "CAT"
'''

pysqldf(query)

The above categories are in the clusters that should be assigned (in theory) to the data. I'm going to do the same for GPS_AVERAGE and MAT_AVERAGE.

In [ ]:
query = '''
SELECT DISTINCT
    GPS_AVERAGE
FROM
    ks2_funding3_df
ORDER BY
    1
'''

ra_df = pysqldf(query)

split_seq(ra_df.values.tolist(),3)

In [ ]:
query = '''
SELECT
    CASE
        WHEN GPS_AVERAGE BETWEEN 91 AND 99 THEN "LOW"
        WHEN GPS_AVERAGE BETWEEN 100 AND 107 THEN "MEDIUM"
        WHEN GPS_AVERAGE > 107 THEN "HIGH"
        ELSE NULL
    END AS CAT,
    COUNT(*)
FROM
    ks2_funding3_df
GROUP BY
    "CAT"
'''

pysqldf(query)

In [ ]:
query = '''
SELECT DISTINCT
    MAT_AVERAGE
FROM
    ks2_funding3_df
ORDER BY
    1
'''

ra_df = pysqldf(query)

split_seq(ra_df.values.tolist(),3)

In [ ]:
query = '''
SELECT
    CASE
        WHEN MAT_AVERAGE BETWEEN 90 AND 98 THEN "LOW"
        WHEN MAT_AVERAGE BETWEEN 99 AND 106 THEN "MEDIUM"
        WHEN MAT_AVERAGE > 106 THEN "HIGH"
        ELSE NULL
    END AS CAT,
    COUNT(*)
FROM
    ks2_funding3_df
GROUP BY
    "CAT"
'''

pysqldf(query)

In [ ]:
query = '''
SELECT
    RECTYPE,
    ALPHAIND,
    LEA,
    ESTAB,
    URN,
    SCHNAME,
    NFTYPE,
    READ_AVERAGE,
    GPS_AVERAGE,
    MAT_AVERAGE,
    "LA Name",
    REGION,
    "REGION NAME",
    _LAESTAB,
    CASE
        WHEN _BALANCE LIKE '%,%' THEN REPLACE(_BALANCE, ',', '')
        ELSE _BALANCE
    END AS _BALANCE,
    CASE
        WHEN READ_AVERAGE BETWEEN 89 AND 98 THEN "LOW"
        WHEN READ_AVERAGE BETWEEN 99 AND 106 THEN "MEDIUM"
        WHEN READ_AVERAGE > 106 THEN "HIGH"
        ELSE NULL
    END AS READ_CAT,
    CASE
        WHEN GPS_AVERAGE BETWEEN 91 AND 99 THEN "LOW"
        WHEN GPS_AVERAGE BETWEEN 100 AND 107 THEN "MEDIUM"
        WHEN GPS_AVERAGE > 107 THEN "HIGH"
        ELSE NULL
    END AS GPS_CAT,
    CASE
        WHEN MAT_AVERAGE BETWEEN 90 AND 98 THEN "LOW"
        WHEN MAT_AVERAGE BETWEEN 99 AND 106 THEN "MEDIUM"
        WHEN MAT_AVERAGE > 106 THEN "HIGH"
        ELSE NULL
    END AS MAT_CAT
FROM
    ks2_funding3_df
'''

ks2_funding3_df = pysqldf(query)

ks2_funding3_df

Now that I have assigned the manual clusters, I will use K-Means to see if the categories match up with the similar numbers.

In [ ]:
kmeans3 = cluster.KMeans(n_clusters=3)

clusteringData_df = ks2_funding3_df[['READ_AVERAGE', '_BALANCE']]

assignedClusters_clust = kmeans3.fit(clusteringData_df)

Counter(assignedClusters_clust.labels_)

In [ ]:
kmeans3.cluster_centers_

In [ ]:
# Plot the data points which in the cluster labelled '0'
plt.scatter(ks2_funding3_df['READ_AVERAGE'][assignedClusters_clust.labels_==0],
            ks2_funding3_df['_BALANCE'][assignedClusters_clust.labels_==0],
            color='red', marker='o', label='cluster 0')

# Plot the data points which in the cluster labelled '1'
plt.scatter(ks2_funding3_df['READ_AVERAGE'][assignedClusters_clust.labels_==1],
            ks2_funding3_df['_BALANCE'][assignedClusters_clust.labels_==1],
            color='blue', marker='o', label='cluster 1')

# Plot the data points which in the cluster labelled '2'
plt.scatter(ks2_funding3_df['READ_AVERAGE'][assignedClusters_clust.labels_==2],
            ks2_funding3_df['_BALANCE'][assignedClusters_clust.labels_==2],
            color='green', marker='o', label='cluster 2')

for (cx, cy) in assignedClusters_clust.cluster_centers_:
    plt.plot(cx, cy, color='black', marker='x', mew=2)

plt.legend()

plt.xlabel('READ_AVERAGE')
plt.ylabel('BALANCE')

plt.title('3-means Clustering with centroids (full dataset)')

plt.plot()

In [ ]:
clusteringDataSample_df = clusteringData_df.sample(n=int(len(clusteringData_df) / 2))

kmeans3 = KMeans(n_clusters=3)
assignedClusters = kmeans3.fit(clusteringDataSample_df)

assignedClusters.labels_


silhouetteData_df = pd.DataFrame({'silhouette':silhouette_samples(clusteringDataSample_df,
                                                                  assignedClusters.labels_),
                                  'cluster':assignedClusters.labels_})

silhouetteData_df.head()

In [ ]:
silhouetteData_df.sort_values(['cluster', 'silhouette'], inplace=True)
silhouetteData_df.index = range(len(silhouetteData_df))

colourMap = {0:'red',
             1:'blue',
             2:'lightGreen'}

for cluster in set(silhouetteData_df['cluster']):
    plt.bar(silhouetteData_df[silhouetteData_df['cluster']==cluster].index,
            silhouetteData_df[silhouetteData_df['cluster']==cluster]['silhouette'],
            color=colourMap[cluster],
            label='Cluster {}'.format(cluster), width=1)

plt.title('Silhouette plot of KS2 READ AVERAGE data, clustered with $k$-means, $k$=3 (50% Sample)')

plt.xlabel('Number of data point')
plt.ylabel('Silhouette coefficient')

plt.legend()

plt.ylim((-0.2, 1.1))

In [ ]:
kmeans3.cluster_centers_

In [ ]:
# Plot the data points which in the cluster labelled '0'
plt.scatter(clusteringDataSample_df['READ_AVERAGE'][assignedClusters.labels_==0],
            clusteringDataSample_df['_BALANCE'][assignedClusters.labels_==0],
            color='red', marker='o', label='cluster 0')

# Plot the data points which in the cluster labelled '1'
plt.scatter(clusteringDataSample_df['READ_AVERAGE'][assignedClusters.labels_==1],
            clusteringDataSample_df['_BALANCE'][assignedClusters.labels_==1],
            color='blue', marker='o', label='cluster 1')

# Plot the data points which in the cluster labelled '2'
plt.scatter(clusteringDataSample_df['READ_AVERAGE'][assignedClusters.labels_==2],
            clusteringDataSample_df['_BALANCE'][assignedClusters.labels_==2],
            color='green', marker='o', label='cluster 2')

for (cx, cy) in assignedClusters_clust.cluster_centers_:
    plt.plot(cx, cy, color='black', marker='x', mew=2)

plt.legend()

plt.xlabel('READ_AVERAGE')
plt.ylabel('BALANCE')

plt.title('3-means Clustering with centroids (50% sample)')

plt.plot()

In [ ]:
Counter(assignedClusters.labels_)